# 1)IMPORT DATA AND SPLIT IT

In [5]:
import pandas as pd
import numpy as np
hit = pd.read_csv("../input/hittlers/Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)

# 2)MODEL FIT

In [6]:
from sklearn.svm import SVR
svr_rbf = SVR("rbf").fit(X_train, y_train)

# 3)PREDICT

In [7]:
import numpy as np
y_pred=svr_rbf.predict(X_test)

# 4) TEST ERROR WITHOUT TUNNING

In [9]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,y_pred))

460.0032657244849

# 5) MODEL TUNNING

In [10]:
svr_rbf

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Here, we want to optimize "C"parameter

In [11]:
svr_params= {"C":[0.1,0.4,5,10,20,30,
               40,50]}

In [12]:
from sklearn.model_selection import GridSearchCV
svr_cv_model= GridSearchCV(svr_rbf,svr_params,cv=10)
svr_cv_model.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.4, 5, 10, 20, 30, 40, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [13]:
svr_cv_model.best_params_

{'C': 50}

In [14]:
svr_tuned_cv_model=SVR("rbf",C=pd.Series(svr_cv_model.best_params_)[0]).fit(X_train,y_train)

In [15]:
y_pred=svr_tuned_cv_model.predict(X_test)

In [16]:
np.sqrt(mean_squared_error(y_test,y_pred))

428.00406888095694

# 428 is test error for SVR tuned model. This is our real test error.
# As you remember, we have found 367 for SVR untuned model(the previous ipynb )

In [ ]:
#Thanks to https://github.com/mvahit/DSMLBC